In [0]:
from pyspark.sql import SparkSession

In [0]:
# Inicializa a sessão Spark
spark = SparkSession.builder.getOrCreate()

In [0]:
def validate_range(table_name: str, condition: str, description: str):
    # Valida se há registros que não atendem à condição de intervalo
    df = spark.table(table_name)
    if df.filter(condition).count() > 0:
        raise Exception(f"[GOLD] Regra inválida ({description}) em {table_name}.")

def validate_not_null(table_name: str, column_name: str):
    # Valida se há valores nulos na coluna especificada
    df = spark.table(table_name)
    if df.filter(f"{column_name} IS NULL").count() > 0:
        raise Exception(f"[GOLD] Coluna {column_name} contém NULL em {table_name}.")

In [0]:
def run_validations():
    # Valida posição dos clubes
    validate_range(
        "project_data_football_gold.classificacao_clubes",
        "posicao < 1 OR posicao > 20",
        "posição fora do intervalo 1-20"
    )

    # Valida aproveitamento dos clubes
    validate_range(
        "project_data_football_gold.classificacao_clubes",
        "aproveitamento_pct < 0 OR aproveitamento_pct > 100",
        "aproveitamento fora do intervalo 0-100"
    )

    # Valida pontos dos clubes
    validate_range(
        "project_data_football_gold.classificacao_clubes",
        "pontos < 0",
        "pontos negativos"
    )

    # Valida média de gols por jogo
    validate_range(
        "project_data_football_gold.kpi_partidas",
        "media_gols_jogos < 0",
        "média de gols negativa"
    )

    # Valida total de jogos nos estádios
    validate_range(
        "project_data_football_gold.kpi_estadios",
        "total_jogos < 0",
        "total de jogos negativo"
    )

    # Valida valores nulos em pontuações
    validate_not_null(
        "project_data_football_gold.kpi_pontuacoes_cartola",
        "valor"
    )

    # Validações concluídas
    print("Validações GOLD concluídas com sucesso.")

if __name__ == "__main__":
    run_validations()